In [9]:
# Interactive Linear Regression with Gradient Descent (Colab-ready, Step-by-Step)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, FloatSlider, IntSlider, VBox, HBox, Output, Layout, HTML, ToggleButton

# Generate simple data
np.random.seed(0)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)
X_b = np.c_[np.ones((100, 1)), X]

# Initialize weights
theta = np.zeros((2, 1))
output = Output()
step_info = HTML()
step_counter = 0

x_min, x_max = 0, 2
y_min, y_max = 0, y.max() + 1

# Widgets
lr_slider = FloatSlider(description='Learning rate', min=0.001, max=0.5, step=0.001, value=0.1)
batch_slider = IntSlider(description='Batch size', min=1, max=100, step=5, value=100)
step_button = Button(description='Step')
reset_button = Button(description='Reset')
toggle_button = ToggleButton(description='Random Init: OFF', value=False)

# Layout styling
lr_slider.layout = Layout(width='300px')
batch_slider.layout = Layout(width='300px')
step_button.layout = Layout(width='150px')
reset_button.layout = Layout(width='150px')
toggle_button.layout = Layout(width='150px')

# Arrange controls
buttons_column = VBox(
    [step_button, reset_button, toggle_button],
    layout=Layout(align_items='center', gap='10px')
)
sliders_column = VBox(
    [lr_slider, batch_slider],
    layout=Layout(align_items='flex-start', gap='10px')
)
controls_column = VBox(
    [buttons_column, sliders_column],
    layout=Layout(align_items='flex-start', gap='20px')
)
outputs = VBox([output, step_info])
ui = VBox([controls_column, outputs], layout=Layout(padding='10px', border='1px solid lightgray'))

# --- Helper to plot ---
def plot_current_line(title=""):
    with output:
        output.clear_output(wait=True)
        plt.figure(figsize=(8, 6))
        plt.scatter(X, y, color="blue", label="Data")
        x_plot = np.linspace(x_min, x_max, 100)
        y_plot = theta[0][0] + theta[1][0] * x_plot
        plt.plot(x_plot, y_plot, color="red", label="Line")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.title(title)
        plt.legend()
        plt.show()

# --- Reset logic ---
def reset_model():
    global theta, step_counter
    if toggle_button.value:
        theta = np.random.randn(2, 1)
    else:
        theta = np.zeros((2, 1))
    step_counter = 0
    plot_current_line(f"Init Intercept: {theta[0][0]:.2f}, Slope: {theta[1][0]:.2f}")
    step_info.value = (
        f"<b>Step:</b> {step_counter} <br>"
        f"<b>Intercept:</b> {theta[0][0]:.4f} &nbsp;&nbsp; <b>Slope:</b> {theta[1][0]:.4f}"
    )

# --- Initial plot ---
reset_model()

# --- Handlers ---
def on_step_clicked(b):
    global theta, step_counter
    m = len(X_b)
    indices = np.random.choice(m, batch_slider.value, replace=False)
    X_batch = X_b[indices]
    y_batch = y[indices]
    old_theta = theta.copy()
    gradients = 2 / batch_slider.value * X_batch.T.dot(X_batch.dot(theta) - y_batch)
    theta -= lr_slider.value * gradients
    step_counter += 1

    plot_current_line(f"Step {step_counter}")

    step_info.value = (
        f"<b>Step:</b> {step_counter} <br>"
        f"<b>Old Intercept:</b> {old_theta[0][0]:.4f} &nbsp;&nbsp; <b>Old Slope:</b> {old_theta[1][0]:.4f} <br>"
        f"<b>Gradient Intercept:</b> {gradients[0][0]:.4f} &nbsp;&nbsp; <b>Gradient Slope:</b> {gradients[1][0]:.4f} <br>"
        f"<b>New Intercept:</b> {theta[0][0]:.4f} &nbsp;&nbsp; <b>New Slope:</b> {theta[1][0]:.4f}"
    )

def on_reset_clicked(b):
    reset_model()

def on_toggle_change(change):
    toggle_button.description = 'Random Init: ON' if toggle_button.value else 'Random Init: OFF'
    reset_model()  # Auto-reset when toggled

step_button.on_click(on_step_clicked)
reset_button.on_click(on_reset_clicked)
toggle_button.observe(on_toggle_change, names='value')

# Display
ui
